In [ ]:
import earthkit as ekt
import pygrib
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature

In [ ]:
model = 'gfs-test'
gribFile = rf'data/gribs/{model.lower()}/latest/{model.lower()}-latest.grb2'
#gribFile = rf'data/gribs/{model.lower()}/maxmin/{model.lower()}-036.grb2'
ds = ekt.data.from_source('file', gribFile)


In [ ]:
grbs = pygrib.open(gribFile)
for grb in grbs:
    print(grb)

In [ ]:
print(ds.ls())

In [ ]:
steps = []
for ts in ds:
    #print(ts.ls())
    #print('name ', ts.metadata('name'), 'param num ', ts.metadata('parameterNumber'))
    #print('step ', ts.metadata('step'))
    step = ts.metadata('step')
    if step not in steps:
        steps.append(step)

print(steps)



In [ ]:
for step in steps:
    ts = ds.sel(step=step, stepType='instant')
    ts = ts.sel(shortName = ['cfrzr', 'csnow', 'cicep', 'refc', 'prmsl'])
    print(ts.ls())

In [ ]:
pressure = ts.sel(shortName = 'prmsl')
refc = ts.sel(shortName = 'refc')
csnow = ts.sel(shortName = 'csnow')[0]
cicep = ts.sel(shortName = 'cicep')[0]
cfrzr = ts.sel(shortName = 'cfrzr')[0]
refc_np = refc.to_numpy()
csnow_np = csnow.to_numpy()
cfrzr_np = cfrzr.to_numpy()
cicep_np = cicep.to_numpy()
refc_snow = np.squeeze(np.where(csnow_np == 1, refc_np, 0) + 100)
refc_ice = np.squeeze(np.where((cfrzr_np == 1) | (cicep_np == 1), refc_np, 0) + 200)


In [ ]:

refc_snow_field = ts[0].clone(values=refc_snow)
refc_ice_field = ts[0].clone(values=refc_ice)

In [ ]:
import matplotlib.colors as mcolors

color_values_refc = {
    5: '#FFFFFF00',
    10: '#00FF0088',
    20: 'green',
    30: 'yellow',
    40: 'orange',
    50: 'red',
    60: 'purple',
    70: 'pink',
    80: 'orange'
}

# Create the normalized bounds (0 to 1) for the colormap
""" norm = mcolors.Normalize(vmin=min(color_values_refc.keys()), vmax=max(color_values_refc.keys()))
# Create a LinearSegmentedColormap
cmap_refc = mcolors.LinearSegmentedColormap.from_list(
    'colormap_refc',
    [(norm(value), color) for value, color in color_values_refc.items()]
) """

norm = mcolors.BoundaryNorm(
    boundaries=list(color_values_refc.keys()),
    ncolors=len(color_values_refc),
    extend='neither' )

values = list(color_values_refc.values())
levels = list(color_values_refc.keys())
print(levels)
print(values)
cmap = mcolors.ListedColormap(values)

norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, extend='neither')

color_values_refc_snow = {
    105: '#FFFFFF00',
    110: '#0000FF88',
    120: 'aqua',
    130: 'blue',
    140: 'purple',
    150: 'pink',
    160: 'white',
    170: 'green',
    180: 'orange'
}

# Create the normalized bounds (0 to 1) for the colormap
""" norm = mcolors.Normalize(vmin=min(color_values_refc.keys()), vmax=max(color_values_refc.keys()))
# Create a LinearSegmentedColormap
cmap_refc = mcolors.LinearSegmentedColormap.from_list(
    'colormap_refc',
    [(norm(value), color) for value, color in color_values_refc.items()]
) """

norm_snow = mcolors.BoundaryNorm(
    boundaries=list(color_values_refc_snow.keys()),
    ncolors=len(color_values_refc_snow),
    extend='neither' )

values_snow = list(color_values_refc_snow.values())
levels_snow = list(color_values_refc_snow.keys())
print(levels_snow)
print(values_snow)
cmap_snow = mcolors.ListedColormap(values_snow)

norm_snow = mcolors.BoundaryNorm(levels_snow, ncolors=cmap_snow.N, extend='neither')

color_values_refc_ice = {
    205: '#FFFFFF00',
    210: '#FF000088',
    220: 'red',
    230: 'yellow',
    240: 'green',
    250: 'brown',
    260: 'white',
    270: 'blue',
    280: 'black'
}

# Create the normalized bounds (0 to 1) for the colormap
""" norm = mcolors.Normalize(vmin=min(color_values_refc.keys()), vmax=max(color_values_refc.keys()))
# Create a LinearSegmentedColormap
cmap_refc = mcolors.LinearSegmentedColormap.from_list(
    'colormap_refc',
    [(norm(value), color) for value, color in color_values_refc.items()]
) """

norm_ice = mcolors.BoundaryNorm(
    boundaries=list(color_values_refc_ice.keys()),
    ncolors=len(color_values_refc_ice),
    extend='neither' )

values_ice = list(color_values_refc_ice.values())
levels_ice = list(color_values_refc_ice.keys())
print(levels_ice)
print(values_ice)
cmap_ice = mcolors.ListedColormap(values_ice)

norm_ice = mcolors.BoundaryNorm(levels_ice, ncolors=cmap_ice.N, extend='neither')

STYLE = ekt.plots.styles.Style(
    colors="Spectral_r",
    levels=range(15, 33),
    #units="celsius",
    ticks=range(15, 33),
)

STYLE = ekt.plots.styles.Style(levels=levels, cmap=cmap, extend='neither')
STYLE_SNOW = ekt.plots.styles.Style(levels=levels_snow, cmap=cmap_snow, extend='neither', legend_style='None')
STYLE_ICE = ekt.plots.styles.Style(levels=levels_ice, cmap=cmap_ice, extend='neither', legend_style='None')

In [ ]:
import os
print(os.getcwd())
shapefile_path = "data/geo/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp"
shapefile_path_lakes = "data/geo/ne_110m_lakes/ne_110m_lakes.shp"
# Read the shapefile using Cartopy's Shapereader
shape_data = shpreader.Reader(shapefile_path)
shape_data_lakes = shpreader.Reader(shapefile_path_lakes)
# Extract features from the shapefile
states = ShapelyFeature(shape_data.geometries(), ccrs.PlateCarree(), facecolor="none", edgecolor="none")
lakes = ShapelyFeature(shape_data_lakes.geometries(), ccrs.PlateCarree(), facecolor="none", edgecolor="none")

In [ ]:
domain_bounds = [-130,-62,22,52]
#domain_bounds = [-140,-60,20,90]
crs = ccrs.PlateCarree()
#chart = earthkit.plots.Map(domain="CONUS")
chart = ekt.plots.Map(domain=domain_bounds, crs=crs)
chart.contourf(refc, style=STYLE)
chart.contourf(refc_snow_field, style=STYLE_SNOW)
chart.plot(refc_ice_field, style=STYLE_ICE)

#chart.contour(pressure, units="hPa")
chart.shapes(states,  transform=ccrs.PlateCarree(), adjust_labels=False, labels=False, facecolor="none",  # No fill color
    edgecolor="black",  # Blue outline
    linewidth=0.5,     # Line thickness
    alpha=0.7)
chart.shapes(lakes,  transform=ccrs.PlateCarree(), adjust_labels=False, labels=False, facecolor="none",  # No fill color
    edgecolor="black",  # Blue outline
    linewidth=0.5,     # Line thickness
    alpha=0.7)
chart.legend(location="bottom")

chart.coastlines()
chart.borders()
#chart.urban_areas()
chart.title()
#chart.gridlines()


chart.show()

In [ ]:
print(dir(chart))

In [ ]:
dsm = ds[12].metadata()
print(dir(dsm))
keys = dsm.keys()
for key in list(keys):
    print(key)

In [ ]:
ds1.metadata('parameterNumber')